In [53]:
from transformers import pipeline
summarizer = pipeline("summarization")

In [54]:
summarizer.model.config.__getattribute__('_name_or_path')

'sshleifer/distilbart-cnn-12-6'

In [55]:
import pandas as pd
df_test = pd.read_csv('data/test.csv')

In [56]:
ref_summaries = list(df_test['summary'])
texts = list(df_test['text'])

In [57]:
import logging
logging.getLogger("transformers").setLevel(logging.WARNING)

In [58]:
texts[0]

"  Consider the blow-up $X$ of $\\mathbb{P}^3$ at 6 points in very general position and the 15 lines through the 6 points. We construct an infinite-order pseudo-automorphism $\\phi_X$ on $X$, induced by the complete linear system of a divisor of degree 13. The effective cone of $X$ has infinitely many extremal rays and hence, $X$ is not a Mori Dream Space. The threefold $X$ has a unique anticanonical section which is a Jacobian K3 Kummer surface $S$ of Picard number 17. The restriction of $\\phi_X$ on $S$ realizes one of Keum's 192 infinite-order automorphisms of Jacobian K3 Kummer surfaces. In general, we show the blow-up of $\\mathbb{P}^n$ ($n\\geq 3$) at $(n+3)$ very general points and certain 9 lines through them is not Mori Dream, with infinitely many extremal effective divisors. As an application, for $n\\geq 7$, the blow-up of $\\overline{M}_{0,n}$ at a very general point has infinitely many extremal effective divisors. "

In [59]:
summarizer(texts[0], max_length=20)

[{'summary_text': ' Threefold $X$ has a unique anticanonical section which is a Jacob'}]

In [60]:
# params = {"min_length":5, "max_length":20, "num_beams":50}
candidate_summaries = []

for i, text in enumerate(texts):
    if i % 50 == 0:
        print(i)
    candidate = summarizer(text, min_length=5, max_length=20)#, **params)
    candidate_summaries.append(candidate[0]['summary_text'])

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950


In [61]:
file = open("zero-shot-summaries.txt", "w")
for s in candidate_summaries:
    file.write(s + "\n")
file.close()

In [62]:
candidate_summaries[:5]

[' Threefold $X$ has a unique anticanonical section which is a Jacob',
 ' New affine algebras A_{\\hbar,\\eta}(\\',
 ' Deep Learning (DL) components within a Case-Based Reasoning (CBR)',
 ' An innovative mechanism allows multifold reconfiguration of mechanical rotation of semiconductor nanoent',
 ' Light scattering by inhomogeneities in the index of refraction of a fluid can']

In [1]:
from datasets import load_metric
metric = load_metric("rouge")

ModuleNotFoundError: No module named 'datasets'

In [64]:
def calc_rouge_scores(candidates, references):
    result = metric.compute(predictions=candidates, references=references, use_stemmer=True)
    result = {key: round(value.mid.fmeasure * 100, 1) for key, value in result.items()}
    return result

In [65]:
calc_rouge_scores(candidate_summaries, ref_summaries)

{'rouge1': 30.3, 'rouge2': 14.0, 'rougeL': 26.1, 'rougeLsum': 26.1}